In [1]:
# 初始化
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt


In [7]:
'''
将点要素空间连接到面要素中，同时实现对一个面包含的点的user_quantity值进行求和
'''

# 读取数据
polygon_gdf = gpd.read_file(r"E:\gis\data\test\clipL4.shp")
points_gdf = gpd.read_file(r"E:\gis\data\test\clip_data.shp")

# 确保坐标系统一致
if polygon_gdf.crs != points_gdf.crs:
    points_gdf = points_gdf.to_crs(polygon_gdf.crs)

# 处理每个面，计算内部点的总和并合并属性
results = []
for idx, polygon_row in polygon_gdf.iterrows():
    current_polygon = polygon_row['geometry']
    points_in_polygon = points_gdf[points_gdf.within(current_polygon)]
    
    if not points_in_polygon.empty:
        sum_value = points_in_polygon['user_quant'].sum()
        polygon_attr = polygon_row.drop('geometry')
            
        # 合并属性（添加前缀避免字段名冲突）
        merged_record = pd.concat([
        pd.Series({'sum_value': sum_value}),
        polygon_attr.add_prefix('polygon_'),
        pd.Series({'point_count': len(points_in_polygon)})  # 添加点计数作为参考
    ])
        results.append({
            'data': merged_record,
            'geometry': current_polygon
        })

# 转换为 GeoDataFrame（使用多边形几何）
merged_df = pd.DataFrame([item['data'] for item in results])
merged_gdf = gpd.GeoDataFrame(
    merged_df,
    geometry=[item['geometry'] for item in results],
    crs=polygon_gdf.crs
)

merged_gdf.to_file(r'data\output.shp', driver='ESRI Shapefile', encoding='utf-8')
print('处理完成')


处理完成


C:\Users\20324\AppData\Local\Temp\ipykernel_19276\4067138483.py:42: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_gdf.to_file(r'data\output.shp', driver='ESRI Shapefile', encoding='utf-8')
e:\Program Files\anaconda\envs\GIS\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'polygon_index' to 'polygon_in'
  ogr_write(
e:\Program Files\anaconda\envs\GIS\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'point_count' to 'point_coun'
  ogr_write(
